In [32]:
from __future__ import absolute_import, division, unicode_literals

from pprint import pformat
from unittest import skip, TestCase

from moz_sql_parser import format, parse
from moz_sql_parser.keywords import join_keywords

EXCEPTION_MESSAGE = """
SQL:         %s
Broken SQL:  %s

JSON:
%s
Broken JSON:
%s
"""

class VerificationException(Exception):
    """exception thrown if anything called by the verify_formatting function raises an exception"""

    def __init__(self, expected_sql, expected_json, new_sql, new_json):
        self.expected_sql = expected_sql
        self.expected_json = expected_json
        self.new_sql = new_sql
        self.new_json = new_json

    def __str__(self):
        res = EXCEPTION_MESSAGE % (self.expected_sql, self.new_sql, pformat(self.expected_json), pformat(self.new_json))
        return res

class TestFormatAndParse(TestCase):

    def verify_formatting(self, expected_sql, expected_json):
        new_sql = ""
        new_json = ""
        try:
            new_sql = format(expected_json)
            new_json = parse(new_sql)
            self.assertEqual(new_json, expected_json)
        except Exception as e:
            raise VerificationException(expected_sql, expected_json, new_sql, new_json)

    def test_two_tables(self):
        expected_sql = "SELECT * from XYZZY, ABC"
        expected_json = {'select': '*', 'from': ['XYZZY', 'ABC']}
        self.verify_formatting(expected_sql, expected_json)

    def test_dot_table_name(self):
        expected_sql = "select * from SYS.XYZZY"
        expected_json = {'select': '*', 'from': 'SYS.XYZZY'}
        self.verify_formatting(expected_sql, expected_json)

    @skip("Not sure why")
    def select_one_column(self):
        expected_sql = "Select A from dual"
        expected_json = {'select': [{'value': 'A'}], 'from': ['dual']}
        self.verify_formatting(expected_sql, expected_json)

    @skip("Still need to figure out how to handle quotes")
    def test_select_quoted_name(self):
        expected_sql = 'Select a "@*#&", b as test."g.g".c from dual'
        expected_json = {'select': [{'name': '@*#&', 'value': 'a'}, {'name': 'test.g.g.c', 'value': 'b'}],
                         'from': 'dual'}
        self.verify_formatting(expected_sql, expected_json)

    def test_select_expression(self):
        expected_sql = "SELECT a + b/2 + 45*c + (2/d) from dual"
        expected_json = {'select': {'value': {'add': ['a', {'div': ['b', 2]}, {'mul': [45, 'c']}, {'div': [2, 'd']}]}},
                         'from': 'dual'}
        self.verify_formatting(expected_sql, expected_json)

    def test_select_underscore_name(self):
        expected_sql = "select _id from dual"
        expected_json = {'select': {'value': '_id'}, 'from': 'dual'}
        self.verify_formatting(expected_sql, expected_json)

    def test_select_dots_names(self):
        expected_sql = "select a.b.c._d from dual"
        expected_json = {'select': {'value': 'a.b.c._d'}, 'from': 'dual'}
        self.verify_formatting(expected_sql, expected_json)

In [33]:
a = TestFormatAndParse()

In [34]:
a.test_select_dots_names()